In [1]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import base64

In [4]:
import cv2

In [5]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, Text, HBox, VBox, Image

In [6]:
from IPython.display import display, clear_output
from IPython.display import HTML

In [7]:
from lib.image import image_data
from lib.analysis import dyssynchrony_analysis

In [8]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 20)

In [9]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [10]:
def load_cv2_image_from_bytes(bytes_):
    return cv2.imdecode(np.frombuffer(bytes_, dtype=np.uint8), cv2.IMREAD_COLOR)

def get_uploader_content(uploader):
    return uploader.value[list(uploader.value)[0]]["content"]

def display_html(html):
    display(widgets.HTML(value=html))
    
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
    
def create_inttext(description, min = None, max = None):
    return widgets.BoundedIntText(
        value=0,
        min = min,
        max = max,
        description=description,
        disabled=False
    )
    
def plot_analysis_result_slice(df,slice_number):
    
    fig, ((ax1, ax2)) = plt.subplots(2, figsize=(20,10))

    ax1.plot(df.loc[slice_number, "amplitudes"])
    ax1.set_title('amplitudes')
    ax1.grid()

    # log y axi
    ax2.plot(df.loc[slice_number, "intensities"])
    ax2.set_title('Intensities')
    ax2.grid()
    
    plt.show()    

def read_progress_img():
    animatedGif = "./assets/progress.gif"
    with open(animatedGif , "rb") as file:
        return file.read()

def create_image(image):
    return Image(
        value=image,
        format='gif',
        width=100,
        height=100)

 
def render_image_visualization(analysis_results):
    render_analysis_results_visualization(analysis_results, 
                                          title = "Peaks",
                                          kind = "image",
                                          download_filename = "complete_cell")


def render_slice_visualization(analysis_results, slice_number):
    render_analysis_results_visualization(analysis_results, 
                                          title = "Slice",
                                          kind = "slices",
                                          download_filename = f"slice_{slice_number}",
                                          slice_number = slice_number)

    
def render_analysis_results_visualization(analysis_results, title, kind, download_filename, slice_number = 0):
    display_html(f"<h3>{title} table</h3>")
    table = build_analysis_results_table(analysis_results, kind = kind)
    table = table[table.slice_number == slice_number]
    table = table.drop(columns = "slice_number")
    display(table)
    
    with_progress(lambda: display(create_download_link(table, title = f"Download {title} Table", filename = f"{download_filename}.csv")))    

    display_html(f"<h3>{title} plot</h3>")
    with_progress(lambda: plot_analysis_result_slice(analysis_results[kind], slice_number))    
    
    
def display_results(analysis_results):
    """
    Displays the results into the results_box. 
    Every print or display will be automatically redirected to it. 
    """
    results_box.clear_output()
    
    display_html("<h2>Complete Cell</h2>")
    render_image_visualization(analysis_results)
    
    display_html("<h2>Slices General Data</h2>")

    table = build_analysis_results_table(analysis_results, kind = 'slices')
    with_progress(lambda: display(
        create_download_link(
            build_analysis_results_table(analysis_results, kind = 'slices'), 
            title = "Download Slices Table", 
            filename = "slices.csv")))    

    display_html("<h2 style='margin-top:20px; margin-botton:15px'>Slices Detail</h2>")
    slice_selector = create_inttext("Plot slice: ", min = 0, max = len(analysis_results["slices"]) - 1)
    plot_btn = widgets.Button(description="Show details", button_style='danger')
    plot_box = widgets.Output()

    
    @plot_box.capture()
    def on_button_clicked(_button):
        plot_box.clear_output()
        render_slice_visualization(analysis_results, slice_selector.value)
        
        
    plot_btn.on_click(on_button_clicked)
    
    display(slice_selector)
    display(plot_btn)
    display(plot_box)

def with_progress(code):
    progress = create_image(read_progress_img())
    display(progress)
    code()
    progress.close()
    
def clear_uploader(uploader):
    uploader.value.clear()
    uploader._counter = 0

def cardiaAp_analyzer(image):
    return dyssynchrony_analysis.analyze_image(image, \
                                               min_dist_between_maxs.value, \
                                               calibration.value, \
                                               slice_width=slice_width.value)

def transpose_results_table(df):
    result = df[["amplitudes", "max_peaks_positions", "max_peaks_intensities", "times_to_peaks",  "times_to_half_peaks", "tau_s"]].apply(pd.Series.explode).reset_index()
    result = result.rename(columns = {"index": "slice_number"})
    result = result.astype({
        "amplitudes":"float", 
        "max_peaks_positions":"int",
        "max_peaks_intensities":"int",
        "times_to_peaks": "int", 
        "times_to_half_peaks": "int",
        "tau_s": "float"
     })
    return result

def build_analysis_results_table(analysis_results, kind):
    """
    Build the final output analysis results table.
    This function works both for building slices table - where you will get a row for each slice and position - 
    or and for building image table - where you get a single row for each position
    Final tables contains the same columns as raw analysis results, except by min_* columns and amplitudes
    """
    return transpose_results_table(last_analysis_results[kind])
    

In [11]:
uploader = widgets.FileUpload(accept='.tif', multiple=False)

slice_width = widgets.IntText(
    value=1,
    description='Slice width:',
    disabled=False
)

min_dist_between_maxs = widgets.IntText(
    value=0,
    description='Dist between maxs:',
    disabled=False
)

calibration = widgets.IntText(
    value=0,
    description='Calibration:',
    disabled=False
)

In [12]:
html_text_upload = "<p style='font-size:15px; color:black'> Upload your image to initialize the analysis</p>"
header_text_upload = widgets.HTML(value=html_text_upload)

box_upload = Layout(display='flex',
                    flex_flow='flex-wrap',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding=' 2px 5px 0 5px',
                    justify_content='space-between')

vbox_upload = widgets.VBox([header_text_upload, uploader], layout=box_upload)

In [13]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding=' 2px 5px 0 5px',
                    justify_content='space-between')

vbox_settings = widgets.VBox([slice_width, min_dist_between_maxs, calibration], layout=box_layout)

In [14]:
analyze_btn = widgets.Button(description="Analyze", button_style='danger')
results_box = widgets.Output()

@results_box.capture()
def on_button_clicked(_button):
    """
    Runs analysis and displays results. 
    It stores them in last_analysis_results
    for debugging purposes
    """
    global last_analysis_results
    results_box.clear_output()

    if uploader._counter == 0:
        print("Please upload an image first")
    else:
        print("Anayzing, please wait...")
        raw_results = cardiaAp_analyzer(load_cv2_image_from_bytes(get_uploader_content(uploader)))
        last_analysis_results = {
            "raw": raw_results,
            "image": pd.DataFrame([raw_results["image"]]),
            "slices":  pd.DataFrame(raw_results["slices"])
        }
        display_results(last_analysis_results)
        clear_uploader(uploader)
        
analyze_btn.on_click(on_button_clicked)

In [15]:
clear_uploader(uploader)
display_html("<h1 style='font-size:25px; color:black'> Welcome to Cardi<b>AP</b> </h1>")

display(vbox_upload)
display(vbox_settings)

display(analyze_btn)
display(results_box)

HTML(value="<h1 style='font-size:25px; color:black'> Welcome to Cardi<b>AP</b> </h1>")

Button(button_style='danger', description='Analyze', style=ButtonStyle())

Output()

In [16]:
# columns = last_analysis_results["slices"].columns

In [17]:
# columns = last_analysis_results["slices"].columns
# for column in columns: 
#     print(column, len(last_analysis_results["slices"].loc[0, column]))